In [ ]:
#| default_exp notation.glossary

# notation.glossary
> Functions for viewing definitions and notations at once

In [ ]:
#| export

from typing import List, Any
from bs4.element import Tag

from trouver.helper.html import remove_html_tags_in_text
from trouver.obsidian.vault import VaultNote
from trouver.notation.in_standard_info_note import notation_notes_linked_in_see_also_section
from trouver.notation.parse import parse_notation_note 

from trouver.personal_vault.notes import (
    notes_linked_in_note, 
    notes_linked_in_notes_linked_in_note
)
from trouver.personal_vault.note_type import note_is_of_type, PersonalNoteTypeEnum

In [ ]:
#| export

#| export
from typing import List, Any
from bs4.element import Tag

# --- Assumed Imports from Your Library ---
# from trouver.obsidian.vault import VaultNote
# from trouver.helper.html import remove_html_tags_in_text
# from trouver.obsidian.links import notation_notes_linked_in_see_also_section
# from trouver.notation.parse import parse_notation_note 

#| export
def generate_glossary_markdown(info_notes: List[VaultNote]) -> str:
    """
    Generates a Markdown-formatted glossary from a list of info notes,
    only including notes that contain at least one definition or notation.

    Args:
        info_notes: A list of VaultNote objects to process.
        
    Returns:
        A single string containing the formatted Markdown glossary.
    """
    glossary_lines = []
    
    for note in info_notes:
        # Temporary list to hold items for the current note
        note_items = []
        
        # 1. Extract Definitions from HTML tags
        _, tags_and_locs = remove_html_tags_in_text(note.text())
        
        for tag, _, _ in tags_and_locs:
            if isinstance(tag, Tag) and tag.has_attr('definition'):
                definition_name = tag['definition']
                if definition_name:
                    note_items.append(f"    - {definition_name}")
        
        # 2. Extract Notations from linked notation notes
        linked_notation_notes = notation_notes_linked_in_see_also_section(note, note.vault)
        
        for notation_note in linked_notation_notes:
            parsed_data = parse_notation_note(notation_note)
            
            latex_str = None
            latex_str = parsed_data.notation_str.strip('$')
            
            if latex_str:
                note_items.append(f"    - ${latex_str}$")
                
        # 3. CRITICAL CHANGE: Only add the note to the glossary if it has items.
        if note_items:
            # Add the note's header first
            glossary_lines.append(f"- [[{note.name}]]")
            # Then add all the collected items
            glossary_lines.extend(note_items)
                
    return "\n".join(glossary_lines)


# def generate_glossary_markdown(info_notes: List[VaultNote]) -> str:
#     """
#     Generates a Markdown-formatted glossary from a list of info notes.

#     For each note, it lists:
#     - Definitions extracted from `definition` attributes in HTML tags.
#     - Notations from associated notation notes, parsed using the existing library function.
    
#     Args:
#         info_notes: A list of VaultNote objects to process.
        
#     Returns:
#         A single string containing the formatted Markdown glossary.
#     """
#     markdown_lines = []
    
#     for note in info_notes:
#         # Add the main info note link as a top-level list item
#         markdown_lines.append(f"- [[{note.name}]]")
        
#         # 1. Extract Definitions from HTML tags
#         _, tags_and_locs = remove_html_tags_in_text(note.text())
        
#         for tag, _, _ in tags_and_locs:
#             if isinstance(tag, Tag) and tag.has_attr('definition'):
#                 definition_name = tag['definition']
#                 if definition_name:
#                     markdown_lines.append(f"    - {definition_name}")
        
#         # 2. Extract Notations from linked notation notes
#         linked_notation_notes = notation_notes_linked_in_see_also_section(
#             note, note.vault)
        
#         for notation_note in linked_notation_notes:
#             # Call the existing parser. 
#             # This returns a structured object (dict or class), NOT a string.
#             parsed_data = parse_notation_note(notation_note)
            
#             # Extract the LaTeX string from the parsed data.
#             # ADJUST THIS KEY based on your actual return structure.
#             # Based on previous context, it might be 'notation', 'latex_in_original', or similar.
#             latex_str = None
            
#             # Example: if the parser returns a dict with a 'notation' key
#             latex_str = parsed_data.notation_str.strip('$')
            
#             # Fallback: if it uses 'latex_in_original' (list or str)
#             # if not latex_str
#             #     val = parsed_data.yaml_frontmatter['latex_in_original']
#             #     latex_str = val[0] if isinstance(val, list) and val else str(val)
            
#             # If parse_notation_note returns an object, access the attribute:
#             # elif hasattr(parsed_data, 'notation'):
#             #     latex_str = parsed_data.notation

#             if latex_str:
#                 markdown_lines.append(f"    - ${latex_str}$")
                        
#     return "\n".join(markdown_lines)


In [ ]:
from fastcore.test import *
from unittest.mock import patch, MagicMock
from bs4 import BeautifulSoup
from typing import List, Any

# --- Mock Infrastructure ---

class MockVaultNote:
    """A simple mock to represent a VaultNote object with a name and text content."""
    def __init__(self, name: str, content: str = ""):
        self.name = name
        self._content = content

    def text(self) -> str:
        return self._content

    def __repr__(self) -> str:
        # Helpful for debugging test failures
        return f"<MockVaultNote name='{self.name}'>"

def make_tag(html_str: str) -> Any:
    """Helper to create a BeautifulSoup Tag object from an HTML string."""
    return BeautifulSoup(html_str, 'html.parser').find()

# --- Tests ---

def test_generate_glossary_basic_case():
    """Tests generating a glossary for a single note with one definition and one notation."""
    # 1. Setup Mock Data
    info_note = MockVaultNote("Info Note 1", "Some content with tags.")
    notation_note = MockVaultNote("notation_x_i", "Content of notation note.")
    
    # Mock data returned by dependencies
    mock_tags = [(make_tag('<b definition="A simple definition">context</b>'), 0, 0)]
    mock_linked_notes = [notation_note]
    mock_parsed_data = {'notation': 'x_i'}

    # 2. Patch dependencies
    with patch('__main__.remove_html_tags_in_text', return_value=("", mock_tags)) as mock_remove_tags, \
         patch('__main__.notation_notes_linked_in_see_also_section', return_value=mock_linked_notes) as mock_find_links, \
         patch('__main__.parse_notation_note', return_value=mock_parsed_data) as mock_parse:
        
        # 3. Run the function
        result = generate_glossary_markdown([info_note])
        
        # 4. Verify
        expected_markdown = (
            "- [[Info Note 1]]\n"
            "    - A simple definition\n"
            "    - $x_i$"
        )
        test_eq(result, expected_markdown)
        
        # Verify dependencies were called correctly
        mock_remove_tags.assert_called_with(info_note.text())
        mock_find_links.assert_called_with(info_note)
        mock_parse.assert_called_with(notation_note)

def test_generate_glossary_multiple_items_and_notes():
    """Tests generating a glossary for multiple notes, each with multiple items."""
    # Note 1 Data
    info_note1 = MockVaultNote("Note A", "Content A")
    def_tag1 = make_tag('<b definition="Def 1">...</b>')
    not_note1 = MockVaultNote("notation_alpha")
    
    # Note 2 Data
    info_note2 = MockVaultNote("Note B", "Content B")
    def_tag2 = make_tag('<b definition="Def 2">...</b>')
    def_tag3 = make_tag('<b definition="Def 3">...</b>')
    not_note2 = MockVaultNote("notation_beta")

    # Mock `remove_html_tags_in_text` to return different tags based on input
    mock_remove_tags_side_effect = {
        info_note1.text(): ("", [(def_tag1, 0, 0)]),
        info_note2.text(): ("", [(def_tag2, 0, 0), (def_tag3, 0, 0)])
    }.get
    
    # Mock `notation_notes_linked_in_see_also_section`
    mock_find_links_side_effect = {
        info_note1: [not_note1],
        info_note2: [not_note2]
    }.get

    # Mock `parse_notation_note`
    mock_parse_side_effect = {
        not_note1: {'notation': r'\alpha'},
        not_note2: {'notation': r'\beta'}
    }.get

    with patch('__main__.remove_html_tags_in_text', side_effect=mock_remove_tags_side_effect), \
         patch('__main__.notation_notes_linked_in_see_also_section', side_effect=mock_find_links_side_effect), \
         patch('__main__.parse_notation_note', side_effect=mock_parse_side_effect):
        
        result = generate_glossary_markdown([info_note1, info_note2])
        
        expected_markdown = (
            "- [[Note A]]\n"
            "    - Def 1\n"
            r"    - $\alpha$"
            "\n"
            "- [[Note B]]\n"
            "    - Def 2\n"
            "    - Def 3\n"
            r"    - $\beta$"
        )
        test_eq(result, expected_markdown)

def test_generate_glossary_no_data():
    """Tests a note with no definitions or linked notations."""
    info_note = MockVaultNote("Empty Note", "No relevant tags or links.")
    
    with patch('__main__.remove_html_tags_in_text', return_value=("", [])), \
         patch('__main__.notation_notes_linked_in_see_also_section', return_value=[]):
        
        result = generate_glossary_markdown([info_note])
        
        # Should only output the note name
        expected_markdown = "- [[Empty Note]]"
        test_eq(result, expected_markdown)

def test_generate_glossary_parser_failure():
    """Tests that if `parse_notation_note` fails (returns None), the notation is skipped."""
    info_note = MockVaultNote("Parser Fail Note", "...")
    notation_note = MockVaultNote("notation_broken")
    
    with patch('__main__.remove_html_tags_in_text', return_value=("", [])), \
         patch('__main__.notation_notes_linked_in_see_also_section', return_value=[notation_note]), \
         patch('__main__.parse_notation_note', return_value=None) as mock_parse: # Simulate failure
        
        result = generate_glossary_markdown([info_note])
        
        # Should not include any notation line
        expected_markdown = "- [[Parser Fail Note]]"
        test_eq(result, expected_markdown)
        mock_parse.assert_called_with(notation_note)

# --- Run Tests ---
# test_generate_glossary_basic_case()
# test_generate_glossary_multiple_items_and_notes()
# test_generate_glossary_no_data()
# test_generate_glossary_parser_failure()


In [ ]:
#| export

#| export
from typing import List, Optional
from pathlib import Path

# --- Assumed Imports from Your Library ---
# from trouver.obsidian.vault import VaultNote
# from trouver.obsidian.personal.notes import (
#     notes_linked_in_note, 
#     notes_linked_in_notes_linked_in_note
# )
# from trouver.obsidian.personal.note_type import is_info_note
# from .glossary import generate_glossary_markdown

def _resolve_info_notes_for_index(index_note: VaultNote) -> List[VaultNote]:
    """
    Determines the list of info notes associated with an index note.
    """
    # One-hop check
    direct_links = notes_linked_in_note(index_note, as_dict=False)
    if direct_links and all(
            note_is_of_type(n, PersonalNoteTypeEnum.STANDARD_INFORMATION_NOTE) for n in direct_links):
        return direct_links

    # Two-hop check
    two_hop_links = notes_linked_in_notes_linked_in_note(
        index_note, as_dict=False)
    if two_hop_links:
        return [n for n in two_hop_links 
                if note_is_of_type(n, PersonalNoteTypeEnum.STANDARD_INFORMATION_NOTE)]

    return []

#| export
def create_glossary_for_index_note(
    index_note: VaultNote,
    info_notes: Optional[List[VaultNote]] = None
) -> None:
    """
    Generates and saves a glossary file for a given index note.

    This function aggregates definitions and notations from a list of associated
    info notes and creates a new `VaultNote` named `_glossary_<name>`
    in the same directory as the index note.

    Args:
        index_note: The `VaultNote` object for the index note.
        info_notes: An optional list of `VaultNote` objects to be included in the
            glossary. If not provided, this function will automatically resolve
            the relevant notes based on the project's linking conventions.
    """
    # 1. Determine the list of info notes to process.
    if info_notes is None:
        info_notes_to_process = _resolve_info_notes_for_index(index_note)
    else:
        info_notes_to_process = info_notes

    if not info_notes_to_process:
        print(f"Warning: No info notes found or provided for index note '{index_note.name}'. No glossary will be generated.")
        return

    # 2. Generate the glossary markdown.
    print(f"Generating glossary for {len(info_notes_to_process)} info note(s)...")
    glossary_content = generate_glossary_markdown(info_notes_to_process)

    # 3. Determine the new note's name and path.
    # Parse the "actually interesting name" using the prefix "_index_"
    prefix = "_index_"
    if index_note.name.startswith(prefix):
        interesting_name = index_note.name[len(prefix):]
    else:
        interesting_name = index_note.name
    
    glossary_name = f"_glossary_{interesting_name}"
    
    # Determine the relative path for the new note.
    # We assume index_note.rel_path is available and is a Path object (or string).
    # The glossary should reside in the same directory.
    parent_dir = Path(index_note.rel_path).parent
    glossary_rel_path = parent_dir / f"{glossary_name}.md"

    # 4. Create the VaultNote object.
    # We assume the VaultNote constructor takes the vault instance and the relative path.
    glossary_note = VaultNote(index_note.vault, rel_path=str(glossary_rel_path))

    # 5. Create the file and write content using VaultNote methods.
    # This ensures proper cache management.
    if not glossary_note.exists():
        glossary_note.create()
    
    # Write the content
    glossary_note.write(glossary_content)
    print(f"Glossary successfully generated: {glossary_note.name}")
